Upload Your Kaggle Json File


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Import the twitter dataset

In [ ]:
!kaggle datasets download kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.42GB/s]


In [ ]:
#extracting compressed dataaset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('the dataset  in extracted')

the dataset  in extracted


Importing the Dependency

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing

In [ ]:
#loading the data from csv file to pandas
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [ ]:
#checking the number of rows and column
twitter_data.shape

(1599999, 6)

In [ ]:
#printing the first five rows of data frame
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#naming the column and reading the dataset again

column_names = ['target', 'id', 'date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names =column_names, encoding = 'ISO-8859-1')

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
#check the distribution of Target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


convert the target "4" to "1"

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 --> Negative Tweet

1 --> Positive Tweet



**Stemming**

stemming is the process of reducing a word to it's root word

example :- actor , actress, acting= act

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def clean_text(text):
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower().split()
    text = [port_stem.stem(w) for w in text if w not in stopwords.words('english') and len(w) > 1]
    return ' '.join(text)

twitter_data['stemmed_content'] = twitter_data['text'].apply(clean_text)


In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",bummer shoulda got david carr third day
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,dive mani time ball manag save rest go bound
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",behav mad see


In [ ]:
print(twitter_data['stemmed_content'])

0                    bummer shoulda got david carr third day
1          upset updat facebook text might cri result sch...
2               dive mani time ball manag save rest go bound
3                            whole bodi feel itchi like fire
4                                              behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996              thewdb com cool hear old walt interview
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999                                 happi charitytuesday
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#seperating the data label
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
print(X)

['bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data to train data and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
#converting the textual data to numerical data
vectorizer = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.8,
    sublinear_tf=True
)

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

model = LogisticRegression(
    solver='liblinear',
    max_iter=1000,
    C=2.0,
    random_state=42
)

model.fit(X_train, Y_train)


LogisticRegression(C=2.0, max_iter=1000, random_state=42, solver='liblinear')

In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10210484 stored elements and shape (1280000, 30000)>
  Coords	Values
  (0, 27827)	0.22628388281644032
  (0, 21242)	0.297687095930213
  (0, 12427)	0.44165948118706727
  (0, 5822)	0.31143585030336
  (0, 14001)	0.3503594636440772
  (0, 28679)	0.37242514278286015
  (0, 27934)	0.5537782682407699
  (2, 5822)	0.3118300367822051
  (2, 6431)	0.15180738391595705
  (2, 25128)	0.15009842056499634
  (2, 6986)	0.23325198614175172
  (2, 24927)	0.2180708859551151
  (2, 27353)	0.26447425313464334
  (2, 3837)	0.251130523567092
  (2, 28690)	0.2686463322261163
  (2, 15802)	0.19375688780115843
  (2, 25351)	0.12171750106790025
  (2, 11986)	0.13092154227556405
  (2, 9287)	0.15085665616533075
  (2, 7275)	0.16280527971341074
  (2, 17077)	0.1347066701570237
  (2, 6482)	0.22401865776843652
  (2, 25535)	0.2826333979259975
  (2, 25003)	0.2388373428855867
  (2, 12042)	0.22133761605350624
  :	:
  (1279996, 17456)	0.46339475059129576
  (1279997, 213)	0.486

In [ ]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2546660 stored elements and shape (320000, 30000)>
  Coords	Values
  (0, 639)	0.13848015180605075
  (0, 1473)	0.13081273559430145
  (0, 1600)	0.33571380864609135
  (0, 3464)	0.21591332110649775
  (0, 5605)	0.2958974546795032
  (0, 7307)	0.20558700851452782
  (0, 7760)	0.19084859971414472
  (0, 7773)	0.29143942259128647
  (0, 11760)	0.22589416354132547
  (0, 16479)	0.1435123870720233
  (0, 16576)	0.273023656941575
  (0, 23889)	0.17707902101553077
  (0, 23892)	0.3390136261045315
  (0, 24519)	0.2810413441842333
  (0, 25351)	0.2142401255789311
  (0, 25378)	0.3390136261045315
  (0, 26634)	0.14471034582743225
  (1, 250)	0.37012212867713745
  (1, 639)	0.25390211519782446
  (1, 12886)	0.49609681063640215
  (1, 15434)	0.34615809495080174
  (1, 21005)	0.5874322114828922
  (1, 22034)	0.29585342730871905
  (2, 1052)	0.37834536882044034
  (2, 1860)	0.4066074650156892
  :	:
  (319995, 10139)	0.18999710751131368
  (319995, 12741)	0.1840526

Trainig Machine Learning Model

Logistic reasoning

Model Evaluation

Accuracy score

In [ ]:
X_train_prediction = model.predict(X_train)
train_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('accuracy score on the training data :' , train_data_accuracy )

accuracy score on the training data : 0.7950875


In [ ]:
#accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('accuracy score on the test data : ', test_data_accuracy)

accuracy score on the test data :  0.783790625


Model accuracy= 82.3%

Saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

# Saveing the vectorizer as well
vectorizer_filename = 'vectorizer.sav'
pickle.dump(vectorizer, open(vectorizer_filename, 'wb'))

In [ ]:
filename= 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the model for future prediction

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [ ]:
X_new=X_test[200]
print(X_new)
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('negative tweet')
else:
  print('positive tweet')

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 12 stored elements and shape (1, 30000)>
  Coords	Values
  (0, 4817)	0.1442558200632725
  (0, 4899)	0.3282564321292121
  (0, 6817)	0.25267558297009407
  (0, 8258)	0.14277390297557263
  (0, 14632)	0.1546332752410735
  (0, 19684)	0.2340819921935262
  (0, 19700)	0.40749854008421516
  (0, 22566)	0.267390822133822
  (0, 22567)	0.36815682202443534
  (0, 23869)	0.360410930036008
  (0, 24560)	0.2567667987645592
  (0, 26918)	0.37361105670192163
1
[1]
positive tweet
